# <font color='blue'> **HW4 Dataset 1**</font>

In [49]:
#install.packages("MLmetrics")
#install.packages("gbm")
library(caret)
library(glmnet)
library(CVXR)
library(MLmetrics)
library(data.table)
library(glmnet)
library(gbm)
library(tidyverse)

### <font color='red'> **Dataset 1: Online News Popularity**</font>

Data Set Information:

* The articles were published by Mashable (www.mashable.com) and their content as the rights to reproduce it belongs to them. Hence, this dataset does not share the original content but some statistics associated with it. The original content be publicly accessed and retrieved using the provided urls.
* Acquisition date: January 8, 2015
* The estimated relative performance values were estimated by the authors using a Random Forest classifier and a rolling windows as assessment method. See their article for more details on how the relative performance values were set.


Attribute Information:

Number of Attributes: 61 (58 predictive attributes, 2 non-predictive, 1 goal field)

Attribute Information:
0. url: URL of the article (non-predictive)
1. timedelta: Days between the article publication and the dataset acquisition (non-predictive)
2. n_tokens_title: Number of words in the title
3. n_tokens_content: Number of words in the content
4. n_unique_tokens: Rate of unique words in the content
5. n_non_stop_words: Rate of non-stop words in the content
6. n_non_stop_unique_tokens: Rate of unique non-stop words in the content
7. num_hrefs: Number of links
8. num_self_hrefs: Number of links to other articles published by Mashable
9. num_imgs: Number of images
10. num_videos: Number of videos
11. average_token_length: Average length of the words in the content
12. num_keywords: Number of keywords in the metadata
13. data_channel_is_lifestyle: Is data channel 'Lifestyle'?
14. data_channel_is_entertainment: Is data channel 'Entertainment'?
15. data_channel_is_bus: Is data channel 'Business'?
16. data_channel_is_socmed: Is data channel 'Social Media'?
17. data_channel_is_tech: Is data channel 'Tech'?
18. data_channel_is_world: Is data channel 'World'?
19. kw_min_min: Worst keyword (min. shares)
20. kw_max_min: Worst keyword (max. shares)
21. kw_avg_min: Worst keyword (avg. shares)
22. kw_min_max: Best keyword (min. shares)
23. kw_max_max: Best keyword (max. shares)
24. kw_avg_max: Best keyword (avg. shares)
25. kw_min_avg: Avg. keyword (min. shares)
26. kw_max_avg: Avg. keyword (max. shares)
27. kw_avg_avg: Avg. keyword (avg. shares)
28. self_reference_min_shares: Min. shares of referenced articles in Mashable
29. self_reference_max_shares: Max. shares of referenced articles in Mashable
30. self_reference_avg_sharess: Avg. shares of referenced articles in Mashable
31. weekday_is_monday: Was the article published on a Monday?
32. weekday_is_tuesday: Was the article published on a Tuesday?
33. weekday_is_wednesday: Was the article published on a Wednesday?
34. weekday_is_thursday: Was the article published on a Thursday?
35. weekday_is_friday: Was the article published on a Friday?
36. weekday_is_saturday: Was the article published on a Saturday?
37. weekday_is_sunday: Was the article published on a Sunday?
38. is_weekend: Was the article published on the weekend?
39. LDA_00: Closeness to LDA topic 0
40. LDA_01: Closeness to LDA topic 1
41. LDA_02: Closeness to LDA topic 2
42. LDA_03: Closeness to LDA topic 3
43. LDA_04: Closeness to LDA topic 4
44. global_subjectivity: Text subjectivity
45. global_sentiment_polarity: Text sentiment polarity
46. global_rate_positive_words: Rate of positive words in the content
47. global_rate_negative_words: Rate of negative words in the content
48. rate_positive_words: Rate of positive words among non-neutral tokens
49. rate_negative_words: Rate of negative words among non-neutral tokens
50. avg_positive_polarity: Avg. polarity of positive words
51. min_positive_polarity: Min. polarity of positive words
52. max_positive_polarity: Max. polarity of positive words
53. avg_negative_polarity: Avg. polarity of negative words
54. min_negative_polarity: Min. polarity of negative words
55. max_negative_polarity: Max. polarity of negative words
56. title_subjectivity: Title subjectivity
57. title_sentiment_polarity: Title polarity
58. abs_title_subjectivity: Absolute subjectivity level
59. abs_title_sentiment_polarity: Absolute polarity level
60. shares: Number of shares (target)

In [2]:
fitControl = trainControl(method = "cv", number = 5)

In [17]:
glm_grid = expand.grid(alpha = 1, lambda = c(0,0.5,1))
dt_grid = expand.grid(cp = c(0.01,0.05,0.1))
rf_grid = expand.grid( mtry = c(3,5))
sgb_grid = expand.grid(n.trees = c(30, 50), shrinkage = c(0.05,0.1), interaction.depth = c(10, 20), n.minobsinnode = 10)

In [4]:
data1 = read.csv("OnlineNewsPopularity.csv",TRUE,",")
data1

url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
http://mashable.com/2013/01/07/amazon-instant-video-browser/,731,12,219,0.6635945,1,0.8153846,4,2,1,...,0.10000000,0.7000000,-0.3500000,-0.6000000,-0.2000000,0.5000000,-0.1875000,0.00000000,0.1875000,593
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/,731,9,255,0.6047431,1,0.7919463,3,1,1,...,0.03333333,0.7000000,-0.1187500,-0.1250000,-0.1000000,0.0000000,0.0000000,0.50000000,0.0000000,711
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/,731,9,211,0.5751295,1,0.6638655,3,1,1,...,0.10000000,1.0000000,-0.4666667,-0.8000000,-0.1333333,0.0000000,0.0000000,0.50000000,0.0000000,1500
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/,731,9,531,0.5037879,1,0.6656347,9,0,1,...,0.13636364,0.8000000,-0.3696970,-0.6000000,-0.1666667,0.0000000,0.0000000,0.50000000,0.0000000,1200
http://mashable.com/2013/01/07/att-u-verse-apps/,731,13,1072,0.4156456,1,0.5408895,19,19,20,...,0.03333333,1.0000000,-0.2201923,-0.5000000,-0.0500000,0.4545455,0.1363636,0.04545455,0.1363636,505
http://mashable.com/2013/01/07/beewi-smart-toys/,731,10,370,0.5598886,1,0.6981982,2,2,0,...,0.13636364,0.6000000,-0.1950000,-0.4000000,-0.1000000,0.6428571,0.2142857,0.14285714,0.2142857,855
http://mashable.com/2013/01/07/bodymedia-armbandgets-update/,731,8,960,0.4181626,1,0.5498339,21,20,20,...,0.10000000,1.0000000,-0.2244792,-0.5000000,-0.0500000,0.0000000,0.0000000,0.50000000,0.0000000,556
http://mashable.com/2013/01/07/canon-poweshot-n/,731,12,989,0.4335736,1,0.5721078,20,20,20,...,0.10000000,1.0000000,-0.2427778,-0.5000000,-0.0500000,1.0000000,0.5000000,0.50000000,0.5000000,891
http://mashable.com/2013/01/07/car-of-the-future-infographic/,731,11,97,0.6701031,1,0.8367347,2,0,0,...,0.40000000,0.8000000,-0.1250000,-0.1250000,-0.1250000,0.1250000,0.0000000,0.37500000,0.0000000,3600
http://mashable.com/2013/01/07/chuck-hagel-website/,731,10,231,0.6363636,1,0.7971014,4,1,1,...,0.10000000,0.5000000,-0.2380952,-0.5000000,-0.1000000,0.0000000,0.0000000,0.50000000,0.0000000,710


In [5]:
data1 = data1[-c(1,2)]

In [6]:
set.seed(1)
trainIndex = createDataPartition(data1$shares, p = 0.9, list = FALSE)
data1_train_1 = data1[trainIndex, ]
data1_test_1 = data1[-trainIndex, ]

In [7]:
set.seed(2)
data1_glmnet =  train(shares ~ ., data = data1_train_1, method = "glmnet", trControl = fitControl, tuneGrid = glm_grid)
data1_glmnet

glmnet 

35680 samples
   58 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 28543, 28545, 28544, 28545, 28543 
Resampling results across tuning parameters:

  lambda  RMSE      Rsquared    MAE     
  0.0     15242.85  0.01924537  3177.163
  0.5     15156.67  0.01927176  3175.303
  1.0     15042.00  0.01929387  3173.089

Tuning parameter 'alpha' was held constant at a value of 1
RMSE was used to select the optimal model using the smallest value.
The final values used for the model were alpha = 1 and lambda = 1.

In [9]:
data1_dt = train(shares ~ ., data = data1_train_1, method = "rpart", trControl = fitControl, tuneGrid = dt_grid)
data1_dt

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
"There were missing values in resampled performance measures."

CART 

35680 samples
   58 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 28543, 28544, 28545, 28544, 28544 
Resampling results across tuning parameters:

  cp    RMSE      Rsquared     MAE     
  0.01  11802.70  0.005642225  3188.899
  0.05  11804.27          NaN  3246.731
  0.10  11804.27          NaN  3246.731

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was cp = 0.01.

In [12]:
set.seed(3)
trainIndex = createDataPartition(data1_train_1$shares, p = 0.15, list = FALSE)
data1_train_11 = data1_train_1[trainIndex, ]
dim(data1_train_11)

[1] 5354   59

In [13]:
data1_rf = train(shares ~ ., data = data1_train_11, method = "rf", trControl = fitControl,tuneGrid = rf_grid)
data1_rf

Random Forest 

5354 samples
  58 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 4283, 4284, 4283, 4282, 4284 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared    MAE     
  3     16681.42  0.01543826  4000.968
  5     16761.51  0.01407119  4095.001

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 3.

In [21]:
data1_sgb = train(shares ~ ., data = data1_train_11, method = "gbm", trControl = fitControl, tuneGrid = sgb_grid)
data1_sgb

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1 235582942.9748             nan     0.0500 146144.9666
     2 232343895.6395             nan     0.0500 237469.9957
     3 231635237.5571             nan     0.0500 318405.1951
     4 230006292.2250             nan     0.0500 218243.6002
     5 228642411.9968             nan     0.0500 -40097.0657
     6 227573497.6083             nan     0.0500 25375.5132
     7 226941723.4601             nan     0.0500 -6892.1875
     8 225625871.5636             nan     0.0500 -194300.0037
     9 225162501.6585             nan     0.0500 29302.5538
    10 223133107.6310             nan     0.0500 84055.0952
    20 214771721.5007             nan     0.0500 -240064.6649
    40 202243207.2697             nan     0.0500 -348948.7385
    50 196394264.5536             nan     0.0500 -180270.4225

Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1 234799349.6508             nan     0.0500 590416.3647
     2 234216404.3546        

Stochastic Gradient Boosting 

5354 samples
  58 predictor

No pre-processing
Resampling: Cross-Validated (5 fold) 
Summary of sample sizes: 4283, 4284, 4282, 4284, 4283 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.trees  RMSE      Rsquared     MAE     
  0.05       10                 30       16899.33  0.004996990  3886.531
  0.05       10                 50       17036.05  0.005100424  3998.072
  0.05       20                 30       16872.37  0.008324167  3902.222
  0.05       20                 50       17000.83  0.007336567  3963.838
  0.10       10                 30       17069.21  0.006793676  4020.975
  0.10       10                 50       17201.58  0.006724476  4175.612
  0.10       20                 30       17163.83  0.006060102  4127.466
  0.10       20                 50       17384.92  0.004330005  4312.278

Tuning parameter 'n.minobsinnode' was held constant at a value of 10
RMSE was used to select the optimal model using the smal

In [36]:
data1_glmnet$bestTune
data1_dt$bestTune
data1_rf$bestTune
data1_sgb$bestTune

,alpha,lambda
3,1,1


cp
0.01


mtry
3


,n.trees,interaction.depth,shrinkage,n.minobsinnode
3,30,20,0.05,10


The best parameters for 4 models are shown above.

In [42]:
glm_RMSE = min(data1_glmnet$result$RMSE)
dt_RMSE = min(data1_dt$result$RMSE)
rf_RMSE = min(data1_rf$result$RMSE)
sgb_RMSE = min(data1_sgb$result$RMSE)
data.table(glm_RMSE,dt_RMSE,rf_RMSE,sgb_RMSE)

glm_RMSE,dt_RMSE,rf_RMSE,sgb_RMSE
15042,11802.7,16681.42,16872.37


As seen the RSME results above, dt gives the best result.

In [43]:
glm_MAE = min(data1_glmnet$result$MAE)
dt_MAE = min(data1_dt$result$MAE)
rf_MAE = min(data1_rf$result$MAE)
sgb_MAE = min(data1_sgb$result$MAE)
data.table(glm_MAE,dt_MAE,rf_MAE,sgb_MAE)

glm_MAE,dt_MAE,rf_MAE,sgb_MAE
3173.089,3188.899,4000.968,3886.531


As seen the MAE result above, glm gives the best result.

#### <font color='purple'> **Summary For Train Set**</font>

For regression dataset, MAE value of GLM closes to the DT values. However, RSME result is the best in DT model. Therefore, DT can be used for train dataset.

In [27]:
data1_glm_predict <- predict(data1_glmnet,data1_test_1)   
data1_dt_predict <- predict(data1_dt,data1_test_1)   
data1_rf_predict <- predict(data1_rf,data1_test_1)   
data1_sgb_predict <- predict(data1_sgb,data1_test_1)   

In [65]:
glm_Test_RMSE = RMSE(data1_test_1$shares,data1_glm_predict)
dt_Test_RMSE = RMSE(data1_test_1$shares,data1_dt_predict)
rf_Test_RMSE = RMSE(data1_test_1$shares,data1_rf_predict)
sgb_Test_RMSE = RMSE(data1_test_1$shares,data1_sgb_predict)
RSME_result = data.table(glm_Test_RMSE,dt_Test_RMSE,rf_Test_RMSE,sgb_Test_RMSE)
RSME_result

glm_Test_RMSE,dt_Test_RMSE,rf_Test_RMSE,sgb_Test_RMSE
7064.095,7153.589,7216.632,7364.056


Contrary to train set result, RMSE value is the best for GLM model. Therefore GLM Model with alpha=1 and lambda=1 is the best for this dataset.